<a href="https://colab.research.google.com/github/ebamberg/research-projects-ml/blob/main/agents_and_routing/examples_agents_with_orchestrator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install ollama langchain_community --quiet
!pip install openai --quiet

host="localhost:11434"
modelid="chevalblanc/gpt-4o-mini"
modelid="deepseek-r1:14b"

get_ipython().system_raw("curl -fsSL https://ollama.com/install.sh | sh")
get_ipython().system_raw("ollama serve &")
get_ipython().system_raw(f"ollama pull {modelid}")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.2 MB/s eta 0:00:00


In [4]:
get_ipython().system_raw(f"ollama pull {modelid}")

In [5]:
from openai import OpenAI


llm = OpenAI(
        base_url=f"http://{host}/v1",
        api_key="ollama",  # required, but unused
    )




In [6]:
from pydantic import Field, BaseModel

class Response(BaseModel):
   action: str = Field(description="the agent to select or \"respond_to_user\" when there are no agents to select from or there is no next_action")
   input: str = Field(description="the input for this agent")
   next_action: str = Field(description="the next agent to call after this agent")


In [7]:
def call(system_prompt: str, message: str, model: str = modelid, history: list[dict] = [] ) -> str:
  completion = llm.chat.completions.parse(
      model=modelid,
      messages=[ {"role": "system", "content": system_prompt},]
      +history
      + [

          {
              "role": "user",
              "content": message,
          },
      ],
      temperature=0.0,
      max_tokens=200,
      response_format=Response
  )

  return completion.choices[0].message.parsed

In [15]:
import json

def orchestrator(userinput: str, memory: list[str] = []) -> Response:


  prompt = """

                You are an intelligent agent orchestrator that coordinates between specialized agents to solve complex problems through multi-step planning and execution.

                Review chat history for context and previous interactions
                Analysise the user's input to classify the intent select the appropriate agent.
                Determine which agents are needed and in what order
                Call the appropriate agent with specific instructions
                Wait for the agent's response
                Evaluate the results

                You will rewrite the input for the agent so that the agent can efficiently execute the task.

                Here are the available agents and their descriptions and input schema:

               current_time_agent - Provides the current time for a given city's timezone like Asia/Kolkata, America/New_York etc. If no timezone is provided, it returns the local time.
               weather_agent - Provides the current weather for a given city.

                User Input:
                {userinput}

                ###Guidelines###
                - Sometimes you might have to use multiple agent's to solve user's input. You have to do that in a loop.
                - The original userinput could have multiple tasks, you will use the context to understand the previous actions taken and the next steps you should take.
                - Read the context, take your time to understand, see if there were many tasks and if you executed them all
                - If there are no actions to be taken, then make the agent "respond_to_user" with your final thoughts combining all previous responses as input.
                - Respond with agent "respond_to_user" when there are no agents to select from or there is no next_action


  """

  for i in range (0,2):

 #   print ("\n".join(memory))
    print (prompt.format(userinput=userinput))
    step = call ("You are an intelligent agent orchestrator that coordinates between specialized agents to solve complex problems through multi-step planning and execution.", prompt.format(userinput=userinput),history=memory)
    print (f"Orchestrator: {step}")
    memory.append({"role": "assistant", "content": step.model_dump_json()},)
    if step.action == "current_time_agent":
      memory.append({"role":"user", "content" :"the current time in New York is 10:00AM in the morning"})
    if step.action == "schedule_agent":
      return orchestrator(step.input)
    if step.action == "weather_agent":
      memory.append({"role":"user", "content" : "The weather in New York is sunny with 39 degress Celsius"})
    if step.action == "respond_to_user":
      return step.input


In [16]:
result = orchestrator ("What is the current time and weather in New York ?")
result



                You are an intelligent agent orchestrator that coordinates between specialized agents to solve complex problems through multi-step planning and execution.

                Review chat history for context and previous interactions
                Analysise the user's input to classify the intent select the appropriate agent.
                Determine which agents are needed and in what order
                Call the appropriate agent with specific instructions
                Wait for the agent's response
                Evaluate the results

                You will rewrite the input for the agent so that the agent can efficiently execute the task.

                Here are the available agents and their descriptions and input schema:

               current_time_agent - Provides the current time for a given city's timezone like Asia/Kolkata, America/New_York etc. If no timezone is provided, it returns the local time.
               weather_agent - Provides the curren

In [10]:
result = orchestrator ("Please schedule a date in my calendar starting in 1 hour.")
result



                You are an expert intent classifier.

                You need will use the context provided and the user's input to classify the intent select the appropriate agent.
                You will rewrite the input for the agent so that the agent can efficiently execute the task.

                Here are the available agents and their descriptions and input schema:

               current_time_agent - Provides the current time for a given city's timezone like Asia/Kolkata, America/New_York etc. If no timezone is provided, it returns the local time.
               weather_agent - Provides the current weather for a given city.

                User Input:
                Please schedule a date in my calendar starting in 1 hour.

                ###Guidelines###
                - Sometimes you might have to use multiple agent's to solve user's input. You have to do that in a loop.
                - The original userinput could have multiple tasks, you will use the context to












A Draft for a first version that used text instead of structured outputs





In [11]:
def orchestrator_text_based(userinput: str) -> str:

  response_format = {"action":"", "input":"", "next_action":""}

  prompt = """
                Use the context from memory to plan next steps.
                Context:
                []

                You are an expert intent classifier.
                You need will use the context provided and the user's input to classify the intent select the appropriate agent.
                You will rewrite the input for the agent so that the agent can efficiently execute the task.

                Here are the available agents and their descriptions:

               time_tool_agent - Provides the current time for a given city's timezone like Asia/Kolkata, America/New_York etc. If no timezone is provided, it returns the local time.
               calendar_agent - Checks your calendar or schedules a date in your calendar at a given time

                User Input:
                {userinput}

                ###Guidelines###
                - Sometimes you might have to use multiple agent's to solve user's input. You have to do that in a loop.
                - The original userinput could have multiple tasks, you will use the context to understand the previous actions taken and the next steps you should take.
                - Read the context, take your time to understand, see if there were many tasks and if you executed them all
                - If there are no actions to be taken, then make the action "respond_to_user" with your final thoughts combining all previous responses as input.
                - Respond with "respond_to_user" only when there are no agents to select from or there is no next_action
                - You will return the agent name in the form of {response_format}
                - Always return valid JSON like {response_format} and nothing else.
  """
  return call ("You are an expert intent classifier.", prompt.format(userinput=userinput, response_format=None))

In [12]:
def orchestrator_(userinput: str, memory: list[str] = []) -> Response:


  prompt = """
                Use the context from memory to plan next steps.
                Context:
                - {context}

                You are an expert intent classifier.

                You need will use the context provided and the user's input to classify the intent select the appropriate agent.
                You will rewrite the input for the agent so that the agent can efficiently execute the task.

                Here are the available agents and their descriptions and input schema:

               current_time_agent - Provides the current time for a given city's timezone like Asia/Kolkata, America/New_York etc. If no timezone is provided, it returns the local time.
               weather_agent - Provides the current weather for a given city.

                User Input:
                {userinput}

                ###Guidelines###
                - Sometimes you might have to use multiple agent's to solve user's input. You have to do that in a loop.
                - The original userinput could have multiple tasks, you will use the context to understand the previous actions taken and the next steps you should take.
                - Read the context, take your time to understand, see if there were many tasks and if you executed them all
                - If there are no actions to be taken, then make the agent "respond_to_user" with your final thoughts combining all previous responses as input.
                - Respond with agent "respond_to_user" when there are no agents to select from or there is no next_action


  """

  for i in range (0,2):

 #   print ("\n".join(memory))
    print (prompt.format(userinput=userinput,context="\n - ".join(memory)))
    step = call ("You are an expert intent classifier.", prompt.format(userinput=userinput,context="\n \t\t- ".join(memory)))
    print (f"Orchestrator: {step}")
    memory.append({"role": "assistant", "content": step},)
    if step.agent_name == "current_time_agent":
      memory.append({"role":"user", "content" :"the current time in New York is 10:00AM in the morning"})
    if step.agent_name == "schedule_agent":
      return orchestrator(step.input)
    if step.agent_name == "weather_agent":
      memory.append({"role":"user", "content" : "The weather in New York is sunny with 39 degress Celsius"})
    if step.agent_name == "respond_to_user":
      return step.input
